Over the last few years, one of the places in SunPy I have been focusing my development efforts has been improving SunPy's coordinate handling. This has really taken two main tracks, the first is the better handling of FITS-WCS in SunPy Map, the second, and the topic of this post, is the representation of physical coordinates in SunPy.
The full story of this post really starts before the 2013 Astropy coordination meeting, which I attended remotely. It was at this meeting I discussed with Erik Tollerud about the future redesign of Astropy's coordinates module and how we could leverage this for SunPy. Skip forward a fair amount and after all the planning and implementing of Astropy coordinates (which I was involved with), we set about implementing the solar physics coordinate systems in SunPy but as a 'plugin' to Astropy's module.

To do this I had the invaluble help of a Goole Summer of Code student [Pritish Chakraborty](http://pritishc.com/) who did the vast majority of the implementation of what is now the `sunpy.coordinates` module. He and I embarked on a mission to learn not just the Astropy coordinates package which we were using, but also all the maths of these systems and the transformations between them, which will be the topic of this post.

Anyway, enough of the back story and onto the coordinates!

## Terminology and Basics

Before we progress any further we must clarify what a coordinate system is and what we are going to call things. A 'coordinate system' is a way of describing points in two dimensional or three dimensional space which is well defined. This 'system' comprises of two main components:

* A representation of a point in vector space, i.e. cartesian, spherical or cylindrical.
* A reference system, which orientates the axes of the system with respect to some well defined points.

We adopt this convention from [APE 5](https://github.com/astropy/astropy-APEs/blob/master/APE5.rst#backgroundterminology).

For the rest of this post we shall mainly be discussing reference systems and the different ones commonly in use in solar physics.

### Projected Coordinate Systems

One common type of coordinate system in astronomy is the projected coordinate system. When observing the sky, the positions of objects on the sky can be described as points on a two dimensional spherical coordinate system, centred upon the observer. This also applies to imaging the sky, as most astronomers do regularly.

Like any other coordinate system a projected system has to have an origin, which is the observer, and it has to be aligned to some reference points. In the case of the solar physics projected coordinate systems these reference points are the solar north pole, and the line from the solar centre to the observer.

There are two solar-specifc projected coordinate systems, helioprojective-cartesian and helioprojective-radial. The first, and most important thing to point out is that, despite the misleading names given to them, they are both fully sperical coordinate systems with positions given as longitude and latitude on the celestial sphere. They are so named, because of their similarities to the true 3D heliocentric coordinate systems we shall discuss later.


## Helioprojective-Cartesian Frame

The most commonly used coordinate frame in solar physics is heliocentric-cartesian, this frame has the zero line of longitude aligned with the line from the solar centre to the solar north pole. It also has the line of zero latitude aligned with the solar equator, meaning that the point of zero longitude and latitude is given to be the centre of the solar disk as seen by the observer.

Due to the small angular size of the Sun as seen from Earth, just under $2000''$ or just over $0.55^\circ$, it has been historically common to approximate this coordinate system as a cartesian system with the y axis being latitude and the x axis being longitude. This is possible due to the small angle approximation, which states that $\sin \theta \approx \theta$ and $\cos \theta \approx 1 - \frac{\theta^2}{2}$ the error in these approximations is well below 1% for any point on the solar disk.

This historical convention has lead to the common labelling of helioprojective longitude and latitude as solar-x and solar-y, and the notion that HPC (helioprojective-cartesian) is actually not a spherical system, which is false.

## Helioprojective-Radial Frame

The helioprojective-radial (HPR) frame is very similar to the helioprojective-cartesian (HPC) frame apart from the point on the celestial sphere that is aligned to the centre of the solar disk. While the HPC system sets the centre of the disk to be the point (0,0) of longitude and latitude, the HPR system defines it to be (0, -90), i.e. at the south pole of the celestial sphere.

As with HPC there is a confusing convention normally adopted when using HPR. By definition the south pole of the celestial sphere actually has the coordinates (0, 0) in HPR and latitude goes from $0^\circ$ to $180^\circ$, rather than from $90^\circ$ to $-90^\circ$ as is usual in spherical coordinates.

In [0]:
%matplotlib nbagg
import matplotlib.pyplot as plt

